<a href="https://www.kaggle.com/code/gabriellacorrea/ativ-3-change-detection?scriptVersionId=239730552" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Uso de Redes Neurais Convolucionais para Detecção de Mudanças

Gabriella Augusta Correa
Luísa Santos


# Importando Bibliotecas

In [1]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch  

# Preparando os dados

In [19]:
class SYSUCDSiameseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        # Filter to keep only files that end with common image extensions
        self.time1_images = sorted([f for f in os.listdir(os.path.join(root_dir, 'time1')) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.time2_images = sorted([f for f in os.listdir(os.path.join(root_dir, 'time2')) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.label_images = sorted([f for f in os.listdir(os.path.join(root_dir, 'label')) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.transform = transform

    def __len__(self):
        return len(self.time1_images)

    def __getitem__(self, idx):
        time1_path = os.path.join(self.root_dir, 'time1', self.time1_images[idx])
        time2_path = os.path.join(self.root_dir, 'time2', self.time2_images[idx])
        label_path = os.path.join(self.root_dir, 'label', self.label_images[idx])
        
        time1_img = Image.open(time1_path).convert("RGB")
        time2_img = Image.open(time2_path).convert("RGB")
        label_img = Image.open(label_path).convert("L")  # Assuming labels are grayscale

        if self.transform:
            time1_img = self.transform(time1_img)
            time2_img = self.transform(time2_img)
            label_img = self.transform(label_img)

        return time1_img, time2_img, label_img

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Adjust the size as needed
    transforms.ToTensor(),
])

# Load datasets
train_dir = '/kaggle/input/modified-sysu-cd/trainmod'
val_dir = '/kaggle/input/modified-sysu-cd/valmod'
test_dir = '/kaggle/input/modified-sysu-cd/testmod'

train_dataset = SYSUCDSiameseDataset(root_dir=train_dir, transform=transform)
val_dataset = SYSUCDSiameseDataset(root_dir=val_dir, transform=transform)
test_dataset = SYSUCDSiameseDataset(root_dir=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Definindo a rede

In [11]:
import torch.nn as nn
import torch

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Shared CNN for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True)  # Upsampling to match input size
        )
        # Output layer for pixel-wise classification
        self.classifier = nn.Conv2d(512, 1, kernel_size=1, stride=1, padding=0)  # 1-channel output for binary mask
        self.sigmoid = nn.Sigmoid()  # Binary mask output

    def forward(self, time1_img, time2_img):
        # Extract features from both images
        feat1 = self.cnn(time1_img)
        feat2 = self.cnn(time2_img)
        
        # Compute absolute difference
        diff = torch.abs(feat1 - feat2)
        
        # Apply classifier to get pixel-wise prediction
        out = self.classifier(diff)
        out = self.sigmoid(out)  # Use sigmoid for binary mask output
        
        return out

# Initialize the model and move it to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)

# Use Binary Cross-Entropy Loss for pixel-wise predictions
criterion = nn.BCELoss()


# Treinando o modelo

In [ ]:
import torch.optim as optim
import torch.nn as nn

# Assuming model, criterion, and optimizer are defined as in previous code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50  # Define your number of epochs

for epoch in range(num_epochs):
    # Training loop
    model.train()
    train_loss = 0.0
    for time1_img, time2_img, label_img in train_loader:
        # Move data to the same device as the model
        time1_img, time2_img, label_img = time1_img.to(device), time2_img.to(device), label_img.to(device)
        
        # Forward pass
        outputs = model(time1_img, time2_img)
        loss = criterion(outputs, label_img.float())
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * time1_img.size(0)  # Accumulate training loss for averaging

    avg_train_loss = train_loss / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for time1_img, time2_img, label_img in val_loader:
            # Move data to the same device as the model
            time1_img, time2_img, label_img = time1_img.to(device), time2_img.to(device), label_img.to(device)
            
            # Forward pass
            outputs = model(time1_img, time2_img)
            loss = criterion(outputs, label_img.float())
            
            val_loss += loss.item() * time1_img.size(0)  # Accumulate validation loss for averaging

    avg_val_loss = val_loss / len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')


# Testando o modelo

## Definindo as funções de teste

In [14]:
# Evaluation function can include IoU calculation
def evaluate(loader, model):
    model.eval()
    with torch.no_grad():
        for time1_img, time2_img, label_img in loader:
            time1_img, time2_img, label_img = time1_img.to(device), time2_img.to(device), label_img.to(device)
            outputs = model(time1_img, time2_img)
            # Convert outputs and label_img to binary for IoU calculation
            # Calculate IoU or any other metric


In [15]:
import time
from sklearn.metrics import f1_score
import numpy as np


In [16]:
def evaluate_model(model, data_loader, device='cpu', threshold=0.5):
    model.eval()
    
    total_latency = 0
    all_f1_scores = []
    all_ious = []

    with torch.no_grad():
        for time1_img, time2_img, label_img in data_loader:
            # Move data to the specified device
            time1_img, time2_img, label_img = time1_img.to(device), time2_img.to(device), label_img.to(device)
            
            # Measure latency
            start_time = time.time()
            outputs = model(time1_img, time2_img)
            latency = (time.time() - start_time) * 1000  # Convert to milliseconds
            total_latency += latency

            # Binarize outputs and labels for F1 and IoU calculation
            preds = (outputs >= threshold).float()
            labels = (label_img >= threshold).float()

            # Compute F1 Score for each batch
            batch_f1 = f1_score(labels.cpu().numpy().flatten(), preds.cpu().numpy().flatten())
            all_f1_scores.append(batch_f1)

            # Compute IoU for each batch
            intersection = (preds * labels).sum().item()
            union = (preds + labels).clamp(0, 1).sum().item()
            iou = intersection / union if union != 0 else 0
            all_ious.append(iou)
    
    # Calculate average metrics
    avg_latency = total_latency / len(data_loader)
    avg_f1_score = np.mean(all_f1_scores)
    avg_iou = np.mean(all_ious)
    
    print(f'Average Latency: {avg_latency:.2f} ms')
    print(f'Average F1 Score: {avg_f1_score:.4f}')
    print(f'Average IoU: {avg_iou:.4f}')
    
    return avg_latency, avg_f1_score, avg_iou


## Testando o modelo

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Evaluate model and print metrics
avg_latency, avg_f1_score, avg_iou = evaluate_model(model, test_loader, device=device)


Average Latency: 1.65 ms
Average F1 Score: 0.5674
Average IoU: 0.4168
